Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.867344
Minibatch accuracy: 18.8%
Validation accuracy: 7.8%
Minibatch loss at step 50: 1.355886
Minibatch accuracy: 50.0%
Validation accuracy: 48.4%
Minibatch loss at step 100: 0.953958
Minibatch accuracy: 68.8%
Validation accuracy: 68.6%
Minibatch loss at step 150: 0.491827
Minibatch accuracy: 75.0%
Validation accuracy: 75.2%
Minibatch loss at step 200: 0.729931
Minibatch accuracy: 81.2%
Validation accuracy: 77.5%
Minibatch loss at step 250: 1.154586
Minibatch accuracy: 62.5%
Validation accuracy: 78.5%
Minibatch loss at step 300: 0.417378
Minibatch accuracy: 87.5%
Validation accuracy: 78.6%
Minibatch loss at step 350: 0.539635
Minibatch accuracy: 93.8%
Validation accuracy: 75.3%
Minibatch loss at step 400: 0.251398
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.822791
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.603653
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [25]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [26]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.009189
Minibatch accuracy: 12.5%
Validation accuracy: 10.2%
Minibatch loss at step 50: 1.690813
Minibatch accuracy: 37.5%
Validation accuracy: 45.2%
Minibatch loss at step 100: 1.284455
Minibatch accuracy: 50.0%
Validation accuracy: 59.5%
Minibatch loss at step 150: 0.405407
Minibatch accuracy: 87.5%
Validation accuracy: 72.6%
Minibatch loss at step 200: 0.898744
Minibatch accuracy: 75.0%
Validation accuracy: 80.5%
Minibatch loss at step 250: 1.177494
Minibatch accuracy: 62.5%
Validation accuracy: 79.3%
Minibatch loss at step 300: 0.374677
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 350: 0.650402
Minibatch accuracy: 93.8%
Validation accuracy: 76.1%
Minibatch loss at step 400: 0.225254
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 450: 0.726629
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 500: 0.651091
Minibatch accuracy: 87.5%
Validation accuracy: 82.9%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [22]:
# meta-parameters
batch_size = 256

# parameters of lenet5
kernel_size_C1 = 5
kernel_size_C3 = 6
depth_C1 = 6
depth_C3 = 16
size_F5 = 120
size_F6 = 84
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # Layer C1 is a convolution layer with 6 feature maps and a 5×5 kernel for each feature map
  layerC1_weights = tf.Variable(tf.truncated_normal(
      [kernel_size_C1, kernel_size_C1, num_channels, depth_C1], stddev=0.1))
  layerC1_biases = tf.Variable(tf.zeros([depth_C1]))
  # Layer C3 is a convolution layer with 16 feature maps and a 6×6 kernel for each feature map
  layerC3_weights = tf.Variable(tf.truncated_normal(
      [kernel_size_C3, kernel_size_C3, depth_C1, depth_C3], stddev=0.1))
  layerC3_biases = tf.Variable(tf.constant(0.1, shape=[depth_C3]))
  # Layer F5 is a fully connected layer of size 120
  layerF5_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth_C3, size_F5], stddev=0.1))
  layerF5_biases = tf.Variable(tf.constant(1.0, shape=[size_F5]))
  # Layer F6 is a fully connected layer of size 84 
  layerF6_weights = tf.Variable(tf.truncated_normal(
          [size_F5, size_F6], stddev=0.1))
  layerF6_biases = tf.Variable(tf.constant(1.0, shape=[size_F6]))
  # output layer
  layerOutput_weights = tf.Variable(tf.truncated_normal(
          [size_F6, num_labels], stddev=0.1))
  layerOutput_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
  # Model.
  def lenet5(data, training=False):
    # Layer C1 is a convolution layer with 6 feature maps and a 5×5 kernel for each feature map
    layerC1 = tf.nn.conv2d(data, layerC1_weights, [1, 1, 1, 1], padding='SAME')
    hiddenC1 = tf.nn.relu(layerC1 + layerC1_biases)
    
    # Layer S2 is a subsampling layer with 6 feature maps and a 2×2 kernel for each feature map
    layerS2 = tf.nn.max_pool(hiddenC1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    # Layer C3 is a convolution layer with 16 feature maps and a 6×6 kernel for each feature map
    layerC3 = tf.nn.conv2d(layerS2, layerC3_weights, [1, 1, 1, 1], padding='SAME')
    hiddenC3 = tf.nn.relu(layerC3 + layerC3_biases)
    
    # Layer S4 is a subsampling layer with 16 feature maps and a 2×2 kernel for each feature map
    layerS4 = tf.nn.max_pool(hiddenC3, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

    # Layer F5 is a fully connected layer with depth 120
    shape = layerS4.get_shape().as_list()
    reshape = tf.reshape(layerS4, [shape[0], shape[1] * shape[2] * shape[3]])
    layerF5 = tf.nn.relu(tf.matmul(reshape, layerF5_weights) + layerF5_biases)
    # during training, add dropout
    if training:
        layerF5 = tf.nn.dropout(layerF5, keep_prob=keep_prob)
    
    # Layer F6 is a fully connected layer with depth 84 
    layerF6 = tf.nn.relu(tf.matmul(layerF5, layerF6_weights) + layerF6_biases)
    
    return tf.matmul(layerF6, layerOutput_weights) + layerOutput_biases
  
  # Training computation.
  logits = lenet5(tf_train_dataset, True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
   
  # learning rate decay
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.80, staircase=True)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(lenet5(tf_valid_dataset))
  test_prediction = tf.nn.softmax(lenet5(tf_test_dataset))

In [23]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.322861
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.610974
Minibatch accuracy: 81.6%
Validation accuracy: 85.6%
Minibatch loss at step 1000: 0.335716
Minibatch accuracy: 88.3%
Validation accuracy: 87.7%
Minibatch loss at step 1500: 0.488528
Minibatch accuracy: 86.7%
Validation accuracy: 87.9%
Minibatch loss at step 2000: 0.383602
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 2500: 0.394785
Minibatch accuracy: 88.3%
Validation accuracy: 88.8%
Minibatch loss at step 3000: 0.439594
Minibatch accuracy: 85.5%
Validation accuracy: 89.3%
Minibatch loss at step 3500: 0.432625
Minibatch accuracy: 87.9%
Validation accuracy: 89.4%
Minibatch loss at step 4000: 0.351330
Minibatch accuracy: 89.5%
Validation accuracy: 89.6%
Minibatch loss at step 4500: 0.373643
Minibatch accuracy: 89.1%
Validation accuracy: 90.0%
Minibatch loss at step 5000: 0.397038
Minibatch accuracy: 87.5%
Validation accurac